#  Creación de la Base de Datos para TFM Cambio Climático
Este notebook define y crea de cero un esquema relacional robusto para almacenar indicadores climáticos, económicos y geográficos.

##  1. Conectar al servidor MySQL (sin base de datos) y asegurar que el esquema exista

En esta celda configuramos la conexión que usaremos para todas las operaciones DDL.

In [5]:
import os
import pymysql
from pymysql.constants import CLIENT

# Cargar variables de entorno
DB_HOST     = os.getenv('DB_HOST')
DB_USER     = os.getenv('DB_USER')
DB_PASSWORD = os.getenv('DB_PASSWORD')
DB_NAME     = os.getenv('DB_NAME')

conexion = pymysql.connect(
    host=DB_HOST,
    user=DB_USER,
    password=DB_PASSWORD,
    client_flag=CLIENT.MULTI_STATEMENTS
)
cursor = conexion.cursor()

# Crear el esquema si no existe
cursor.execute(f"""
  CREATE DATABASE IF NOT EXISTS `{DB_NAME}` 
    CHARACTER SET utf8mb4 
    COLLATE utf8mb4_unicode_ci;
""")
conexion.commit()

# Apuntamos a ese esquema para las siguientes operaciones
cursor.execute(f"USE `{DB_NAME}`;")
conexion.select_db(DB_NAME)


## 2. Creación de las tablas
En esta sección creamos todas las tablas de nuestro modelo relacional reforzado.  
Cada tabla que referencia una dimensión incluye un índice explícito sobre la FK y las cláusulas `ON UPDATE CASCADE` / `ON DELETE RESTRICT` para garantizar integridad y rendimiento.

In [6]:

# Eliminar si existía y crear la base de datos
cursor.execute("SET FOREIGN_KEY_CHECKS=0;")
for tbl in ["Hechos", "Paises", "Continentes", "Indicadores",
            "Categorias", "Unidades"]:
    cursor.execute(f"DROP TABLE IF EXISTS `{tbl}`;")

cursor.execute("SET FOREIGN_KEY_CHECKS=1;")
# 1) Unidades
sql_unidades = """
CREATE TABLE IF NOT EXISTS Unidades (
  id INT AUTO_INCREMENT PRIMARY KEY,
  descripcion VARCHAR(64) NOT NULL UNIQUE
) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_unicode_ci;
"""
cursor.execute(sql_unidades)

# 2) Categorias
sql_categorias = """
CREATE TABLE IF NOT EXISTS Categorias (
  id INT AUTO_INCREMENT PRIMARY KEY,
  nombre VARCHAR(64) NOT NULL UNIQUE
) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_unicode_ci;
"""
cursor.execute(sql_categorias)

# 3) Indicadores
sql_indicadores = """
CREATE TABLE IF NOT EXISTS Indicadores (
  id INT AUTO_INCREMENT PRIMARY KEY,
  codigo VARCHAR(32) NOT NULL UNIQUE,
  descripcion VARCHAR(255),
  unidad_id INT    NOT NULL,
  categoria_id INT NOT NULL,
  INDEX idx_ind_unidad    (unidad_id),
  INDEX idx_ind_categoria (categoria_id),
  CONSTRAINT fk_ind_unidad
    FOREIGN KEY (unidad_id)
    REFERENCES Unidades(id)
    ON UPDATE CASCADE
    ON DELETE RESTRICT,
  CONSTRAINT fk_ind_categoria
    FOREIGN KEY (categoria_id)
    REFERENCES Categorias(id)
    ON UPDATE CASCADE
    ON DELETE RESTRICT
) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_unicode_ci;
"""
cursor.execute(sql_indicadores)


# 4) Continentes
sql_continentes = """
CREATE TABLE IF NOT EXISTS Continentes (
  id INT PRIMARY KEY,
  nombre VARCHAR(64) NOT NULL UNIQUE
) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_unicode_ci;
"""
cursor.execute(sql_continentes)

# 5) Paises
sql_paises = """
CREATE TABLE IF NOT EXISTS Paises (
  codigo VARCHAR(10) PRIMARY KEY,
  nombre VARCHAR(100) NOT NULL,
  nombre_en VARCHAR(100) NOT NULL,
  continente_id INT NOT NULL,
  INDEX idx_pais_continente (continente_id),
  CONSTRAINT fk_pais_continente
    FOREIGN KEY (continente_id)
    REFERENCES Continentes(id)
    ON UPDATE CASCADE
    ON DELETE RESTRICT
) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_unicode_ci;
"""
cursor.execute(sql_paises)

# 6) Hechos
sql_hechos = """
CREATE TABLE IF NOT EXISTS Hechos (
  anio INT NOT NULL,
  valor DOUBLE,
  pais_id VARCHAR(10) NOT NULL,
  indicador_id INT NOT NULL,
  PRIMARY KEY (pais_id, anio, indicador_id),
  INDEX idx_hec_pais      (pais_id),
  INDEX idx_hec_indicador (indicador_id),
  CONSTRAINT fk_hec_pais
    FOREIGN KEY (pais_id)
    REFERENCES Paises(codigo)
    ON UPDATE CASCADE
    ON DELETE RESTRICT,
  CONSTRAINT fk_hec_indicador
    FOREIGN KEY (indicador_id)
    REFERENCES Indicadores(id)
    ON UPDATE CASCADE
    ON DELETE RESTRICT
) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_unicode_ci;
"""
cursor.execute(sql_hechos)

conexion.commit()
print("✅ Todas las tablas han sido creadas correctamente en el esquema.") 


✅ Todas las tablas han sido creadas correctamente en el esquema.


## 3. Insertar datos genéricos

In [7]:
# 3.1 Insertar Categorías
categorias = ['Ambiental', 'Economía', 'Demografía','Catástrofes naturales']
sql_cat = "INSERT INTO Categorias (nombre) VALUES (%s);"
cursor.executemany(sql_cat, [(c,) for c in categorias])
conexion.commit()

# 3.2 Insertar Unidades
unidades = [
    ('Tasa de emisiones de CO₂ (Gg CO2e)',),
    ('Grados Celsius (ºC)',),
    ('Dólar estadounidense ($)',),
    ('Habitantes',),
    ('kg CO₂e por US$ 2021 PPA',),
    ('Porcentaje del total (%)',),
    ('Kilovatio-hora (kWh)',),
    ('Kilómetro cuadrado (km²)',),
    ('Número de desastres',),
    ('Número de personas afectadas',)
]
sql_uni = "INSERT INTO Unidades (descripcion) VALUES (%s);"
cursor.executemany(sql_uni, unidades)
conexion.commit()

# 3.3 Obtener mapeos de PKs de Unidades y Categorías
cursor.execute("SELECT id, descripcion FROM Unidades;")
unidad_map = { desc: uid for uid, desc in cursor.fetchall() }

cursor.execute("SELECT id, nombre FROM Categorias;")
categoria_map = { nombre: cid for cid, nombre in cursor.fetchall() }

# 3.4 Insertar Indicadores
indicadores = [
    # código, descripción, unidad_desc, categoría
    ('temperaturas', 'Temperatura media', 'Grados Celsius (ºC)', 'Ambiental'),
    ('emision_co2', 'Emisión de CO₂', 'Tasa de emisiones de CO₂ (Gg CO2e)', 'Ambiental'),
    ('EN.GHG.CO2.RT.GDP.PP.KD', 'Intensidad de carbono del PIB (PPA 2021)', 'kg CO₂e por US$ 2021 PPA', 'Economía'),
    ('EN.GHG.CO2.RT.GDP.KD', 'Intensidad de carbono del PIB (US$ constantes 2021)', 'kg CO₂e por US$ 2021 PPA', 'Economía'),
    ('EG.ELC.FOSL.ZS', 'Producción eléctrica a partir de fuentes fósiles', 'Porcentaje del total (%)', 'Economía'),
    ('EG.ELC.RNWX.ZS', 'Producción eléctrica de fuentes renovables (excluyendo hidroeléctrica)', 'Porcentaje del total (%)', 'Economía'),
    ('EG.ELC.RNWX.KH', 'Producción eléctrica de fuentes renovables (excluyendo hidroeléctrica)', 'Kilovatio-hora (kWh)', 'Economía'),
    ('EG.ELC.RNEW.ZS', 'Producción de electricidad renovable', 'Porcentaje del total (%)', 'Economía'),
    ('NY.GDP.MKTP.CD', 'Producto Interno Bruto (PIB)', 'Dólar estadounidense ($)', 'Economía'),
    ('AG.LND.TOTL.K2', 'Superficie terrestre total (excluye aguas interiores y ZEE)', 'Kilómetro cuadrado (km²)', 'Demografía'),
    ('SP.POP.TOTL', 'Población total (estimaciones de mitad de año)', 'Habitantes', 'Demografía'),
    ('desastres_sequia', 'Número de sequías', 'Número de desastres', 'Catástrofes naturales'),
    ('desastres_temp_extrema', 'Número de olas de calor / frío extremo', 'Número de desastres', 'Catástrofes naturales'),
    ('desastres_inundacion', 'Número de inundaciones', 'Número de desastres', 'Catástrofes naturales'),
    ('desastres_deslizamiento', 'Número de deslizamientos', 'Número de desastres', 'Catástrofes naturales'),
    ('desastres_tormenta', 'Número de tormentas', 'Número de desastres', 'Catástrofes naturales'),
    ('desastres_total', 'Número total de desastres', 'Número de desastres', 'Catástrofes naturales'),
    ('desastres_incendios', 'Número de incendios forestales', 'Número de desastres', 'Catástrofes naturales'),
    ('afectados_sequia', 'Personas afectadas por sequías', 'Número de personas afectadas', 'Catástrofes naturales'),
    ('afectados_temp_extrema', 'Personas afectadas por calor / frío extremo', 'Número de personas afectadas', 'Catástrofes naturales'),
    ('afectados_inundacion', 'Personas afectadas por inundaciones', 'Número de personas afectadas', 'Catástrofes naturales'),
    ('afectados_deslizamiento', 'Personas afectadas por deslizamientos', 'Número de personas afectadas', 'Catástrofes naturales'),
    ('afectados_tormenta', 'Personas afectadas por tormentas', 'Número de personas afectadas', 'Catástrofes naturales'),
    ('afectados_total', 'Personas afectadas en total', 'Número de personas afectadas', 'Catástrofes naturales'),
    ('afectados_incendios', 'Personas afectadas por incendios forestales', 'Número de personas afectadas', 'Catástrofes naturales'),
]

# Construir data_ind con comprobaciones
data_ind = []
faltan_unidades = set()
faltan_categorias = set()

for code, desc, uni_desc, cat_name in indicadores:
    uid = unidad_map.get(uni_desc)
    cid = categoria_map.get(cat_name)
    if uid is None:
        faltan_unidades.add(uni_desc)
        continue
    if cid is None:
        faltan_categorias.add(cat_name)
        continue
    data_ind.append((code, desc, uid, cid))

if faltan_unidades:
    print("⚠️ Unidades no encontradas en la tabla Unidades:", faltan_unidades)
if faltan_categorias:
    print("⚠️ Categorías no encontradas en la tabla Categorias:", faltan_categorias)

# Inserción (idempotente si 'codigo' es UNIQUE o PK)
sql_ind = """
INSERT INTO Indicadores (codigo, descripcion, unidad_id, categoria_id)
VALUES (%s, %s, %s, %s)
ON DUPLICATE KEY UPDATE
  descripcion = VALUES(descripcion),
  unidad_id   = VALUES(unidad_id),
  categoria_id= VALUES(categoria_id);
"""

if data_ind:
    cursor.executemany(sql_ind, data_ind)
    conexion.commit()
    print(f"Indicadores insertados/actualizados: {len(data_ind)}")
else:
    print("No hay indicadores para insertar (revisa unidades/categorías faltantes).")

print("✅ Inserción de datos genéricos completada.")

Indicadores insertados/actualizados: 25
✅ Inserción de datos genéricos completada.


## 4. Cierre de conexión

In [8]:
cursor.close()
conexion.close()
print("Conexión cerrada.")

Conexión cerrada.
